<a href="https://colab.research.google.com/github/tomandcharliegoonanadventure/University-Creations/blob/main/SQL_Assignment_Tom_OConnor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##

# Assignment 3 158739

#### Student name: Thomas O'Connor
#### Student ID: 05065275

## Introduction
This assignment seeks to put some data into databases. In this assignment I'll get the data from assignment 4, make smaller dataframes out of it, and put them into tables in a database. I'll then request for information from the database and create some views.

This might be becuase i'm new to using SQL, but it seems like it is a language that whilst it's easy to use, is not as fun as python. I feel that with python the only thing limiting what you can do with it is your imagination. SQL is not that, it is solely used for requesting data from data bases.

### Datasets used:
Some New Zealand Incomes by Industries. These are salaries and rather clean data.

An API from a job website in the USA. This data is fairly nice, it has top and bottom expected salaries.

OECD Tax Data. This was from the OECD website that allowed me to download a .csv file which I fed into a dataframe. Lots of data here.

### Dataset sources:
https://figure.nz/chart/hdNat68mEUPexBqH

https://developer.adzuna.com/docs/search

https://stats.oecd.org/Index.aspx?DataSetCode=REV#


In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Let's create a Database

So this was pretty easy to set up. As I don't know how to do this, I've looked it up on SQL Lite's website. Below is the code to achieve a new database in the file that I have this one (The Jupyter Notebook) stored.

In [ ]:
def create_sqlite_database(filename):
    """ create a database connection to an SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(filename)
        print(sqlite3.sqlite_version)
    except sqlite3.Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_sqlite_database("usanzgdp.db")

3.35.5


Below I'm connecting up to the database, and below that I'm importing a dataframe with all the data from assignment 4.

In [ ]:
%sql sqlite:///./usanzgdp.db3

In [ ]:
df= pd.read_csv('./concatdf.csv')
df.head()

C:\Users\crump\AppData\Local\Temp\ipykernel_18916\1701778563.py:1: DtypeWarning: Columns (0,1,4,7,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('./concatdf.csv')


,Job Title,Location,Salary Min,Salary Max,Company,GOV,Level of government,TAX,Tax revenue,VAR,...,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags,Sector,Highest Qualification,NZD Per Annum
0,Senior Software Engineer,"De Pere, Brown County",91064.38,91064.38,OPW,SUPRA,Supranational,TOTALTAX,Total tax revenue,TAXGDP,...,0,Units,NaN,NaN,0.176,NaN,NaN,ICT,No qualification,62064.0
1,Software Engineer I,"Reading Station, Berks County",73028.40,73028.40,Penske Truck Leasing,SUPRA,Supranational,TOTALTAX,Total tax revenue,TAXGDP,...,0,Units,NaN,NaN,0.206,NaN,NaN,ICT,Level 1-4 certificate,72300.0
2,Staff Software Architect,"Reading Station, Berks County",85990.89,85990.89,Penske Truck Leasing,SUPRA,Supranational,TOTALTAX,Total tax revenue,TAXGDP,...,0,Units,NaN,NaN,0.206,NaN,NaN,ICT,Level 5 and 6 diploma,76727.0
3,Software Test Technician,"Keene, Cheshire County",71529.01,71529.01,Markem-Imaje,SUPRA,Supranational,TOTALTAX,Total tax revenue,TAXGDP,...,0,Units,NaN,NaN,0.208,NaN,NaN,ICT,Bachelor degree and level 7 qualification,68903.0
4,Lead Software Engineer - Full Stack,"Bannockburn, Lake County",184085.84,184085.84,WALGREENS,SUPRA,Supranational,TOTALTAX,Total tax revenue,TAXGDP,...,0,Units,NaN,NaN,0.180,NaN,NaN,ICT,"Post-graduate, honours degree or higher qualif...",74809.0


### Check the Connection and Turn on Foreign Keys

Aaaaand I'm checking the connection again. Also turning on foreign keys.

In [ ]:
%sql sqlite:///./usanzgdp.db3 PRAGMA foreign_keys = ON;

Done.


[]

### Create a Pandas Dataframe

So now I'm creating a dataframe and cleaning it to go into the database as a table. I've chosen to use "Year" as a key. Which is pretty neat. I've included the "pretty neat" video link on youtube in the references if you want to have a look. It's the first one.

In [ ]:
usajobsdf = df[['Year','Location','Salary Min','Salary Max']].copy()
usajobsdf = usajobsdf.drop_duplicates(['Year'])
usajobsdf.rename(columns={'Salary Min': 'SalaryMin', 'Salary Max': 'SalaryMax','Year':'Yearjobs'}, inplace=True)
usajobsdf=usajobsdf.set_index('Yearjobs')
usajobsdf=usajobsdf.dropna()
usajobsdf.head()

,Location,SalaryMin,SalaryMax
Yearjobs,,,
1995,"De Pere, Brown County",91064.38,91064.38
1996,"Reading Station, Berks County",73028.40,73028.40
1997,"Reading Station, Berks County",85990.89,85990.89
1998,"Keene, Cheshire County",71529.01,71529.01
1999,"Bannockburn, Lake County",184085.84,184085.84


### Deleting the Table if it Exists.

You can skip this step, but if you need to run anything again it errors out every time. For this notebook I'll be deleting the table if it exists every time because if I go away from the computer, the kernel gets disrupted I need to run everything again. Which means I need to delete the table every time.

In [ ]:
%%sql sqlite:///./usanzgdp.db3
DROP TABLE IF EXISTS USAJOBS;

Done.


[]

### Creating the Table

Below I'm creating a table. I'm telling the computer I would like to have a table with certain columns and please expect certain types of data. Like floats, or integers or whatever.

Then I'm creating an index for this table. I'm using 'Year' as my primary key throughout.

In [ ]:
%%sql sqlite:///./usanzgdp.db3
CREATE TABLE USAJOBS(
    Location VARCHAR(10) NOT NULL,
    SalaryMin FLOAT NOT NULL,
    SalaryMax FLOAT NOT NULL,
    Yearjobs INTEGER NOT NULL UNIQUE,
    PRIMARY KEY (Yearjobs)
);

CREATE INDEX ix_usajobs_year ON USAJOBS (Yearjobs);

Done.
Done.


[]

### Enabling foreign keys

In [ ]:
%sql sqlite:///./usanzgdp.db3 PRAGMA foreign_keys

Done.


foreign_keys
1


### Connecting to the Database

In [ ]:
connection = sqlite3.connect('./usanzgdp.db3')

### Pandas Dataframe to SQL Database

Below I'm committing the pandas dataframe to the SQL database. It is entered as the table I created above.

In [ ]:
usajobsdf.to_sql('USAJOBS', connection, if_exists='append',)
connection.commit()

### Closing the Connection

Below I'm closing the connection to the database. Job done! Time for a coffee.

After this I'm going to be repeating this 5 more times. They are going to be remarkably similar, and I'm not going to explain everything. So you don't get too bored of me writing stuff.

In [ ]:
connection.close()

### Checking the Table in the Database

Below I'm asking SQL to show me what's in the table that I wrote above. I'm doing this to check that it put it in the table. Or it gets the hose again (referenced).

In [ ]:
usajobssqldf = %sql SELECT * FROM USAJOBS
usajobssqldf

 * sqlite:///./usanzgdp.db3
Done.


Location,SalaryMin,SalaryMax,Yearjobs
"Bannockburn, Lake County",194521.14,194521.14,1970
"Reading Station, Berks County",104078.76,104078.76,1971
"Henrietta, Monroe County",110828.35,110828.35,1972
"De Pere, Brown County",101236.55,101236.55,1973
"Juno Beach, Palm Beach County",92076.77,92076.77,1974
"Juno Beach, Palm Beach County",91954.79,91954.79,1975
"Houston, Harris County",63189.18,63189.18,1976
"Mooresville, Limestone County",130763.43,130763.43,1977
"Redstone Arsenal, Madison County",121640.6,121640.6,1978
"Capshaw, Limestone County",121335.3,121335.3,1979


In [ ]:
usatax = df[['Year','Level of government','Value']].copy()
usatax.head()

,Year,Level of government,Value
0,1995,Supranational,0.176
1,1996,Supranational,0.206
2,1997,Supranational,0.206
3,1998,Supranational,0.208
4,1999,Supranational,0.180


In [ ]:
usatax.rename(columns={'Level of government': 'Levelofgovernment','Year':'Yeartax'}, inplace=True)
usatax

,Yeartax,Levelofgovernment,Value
0,1995,Supranational,0.176
1,1996,Supranational,0.206
2,1997,Supranational,0.206
3,1998,Supranational,0.208
4,1999,Supranational,0.180
...,...,...,...
699688,2018,State/Regional,0.000
699689,2019,State/Regional,0.000
699690,2020,State/Regional,0.000
699691,2021,State/Regional,0.000


In [ ]:
usatax = usatax.drop_duplicates(['Yeartax'])
usatax.count()

Yeartax              58
Levelofgovernment    58
Value                58
dtype: int64

In [ ]:
usatax = usatax.set_index('Yeartax')
usatax = usatax.dropna()
usatax.head()

,Levelofgovernment,Value
Yeartax,,
1995,Supranational,0.176
1996,Supranational,0.206
1997,Supranational,0.206
1998,Supranational,0.208
1999,Supranational,0.180


In [ ]:
%%sql sqlite:///./usanzgdp.db3
DROP TABLE IF EXISTS USATAX;

Done.


[]

In [ ]:
%%sql sqlite:///./usanzgdp.db3
CREATE TABLE USATAX(
    LEVELOFGOVERNMENT VARCHAR(10) NOT NULL,
    VALUE FLOAT NOT NULL,
    Yeartax INTEGER NOT NULL,
    PRIMARY KEY (Yeartax)
);

CREATE INDEX ix_usatax_year ON USATAX (Yeartax);

Done.
Done.


[]

In [ ]:
connection = sqlite3.connect('./usanzgdp.db3')
usatax.to_sql('USATAX', connection, if_exists='append',)
connection.commit()

In [ ]:
connection.close()

In [ ]:
usataxsqldf = %sql SELECT * FROM USATAX
usataxsqldf

 * sqlite:///./usanzgdp.db3
Done.


LEVELOFGOVERNMENT,VALUE,Yeartax
Federal or Central government,0.0,1965
Federal or Central government,0.0,1966
Federal or Central government,0.0,1967
Federal or Central government,0.0,1968
Federal or Central government,0.0,1969
Supranational,0.015,1970
Supranational,0.43,1971
Supranational,0.484,1972
Supranational,0.487,1973
Supranational,0.504,1974


In [ ]:
usajobtitledf = df[['Year','Location','Job Title','Company']].copy()
usajobtitledf = usajobtitledf.drop_duplicates(['Year'])
usajobtitledf.rename(columns={'Job Title': 'JobTitle','Year':'Yearjobtitle'}, inplace=True)
usajobtitledf=usajobtitledf.set_index('Yearjobtitle')
usajobtitledf=usajobtitledf.dropna()
usajobtitledf.head()

,Location,JobTitle,Company
Yearjobtitle,,,
1995,"De Pere, Brown County",Senior Software Engineer,OPW
1996,"Reading Station, Berks County",Software Engineer I,Penske Truck Leasing
1997,"Reading Station, Berks County",Staff Software Architect,Penske Truck Leasing
1998,"Keene, Cheshire County",Software Test Technician,Markem-Imaje
1999,"Bannockburn, Lake County",Lead Software Engineer - Full Stack,WALGREENS


In [ ]:
%%sql sqlite:///./usanzgdp.db3
DROP TABLE IF EXISTS USAJOBTITLE;

Done.


[]

In [ ]:
%%sql sqlite:///./usanzgdp.db3
CREATE TABLE USAJOBTITLE(
    LOCATION VARCHAR(10) NOT NULL,
    JOBTITLE VARCHAR(10) NOT NULL,
    COMPANY VARCHAR(10) NOT NULL,
    Yearjobtitle INTEGER NOT NULL,
    PRIMARY KEY (Yearjobtitle)
);

CREATE INDEX ix_usajobtitle_year ON USAJOBTITLE (Yearjobtitle);

Done.
Done.


[]

In [ ]:
connection = sqlite3.connect('./usanzgdp.db3')
usajobtitledf.to_sql('USAJOBTITLE', connection, if_exists='append',)
connection.commit()

In [ ]:
connection.close()

In [ ]:
usajobtitlesqldf = %sql SELECT * FROM USAJOBTITLE
usajobtitlesqldf

 * sqlite:///./usanzgdp.db3
Done.


LOCATION,JOBTITLE,COMPANY,Yearjobtitle
"Bannockburn, Lake County",Senior Software Engineer - Android Developer,WALGREENS,1970
"Reading Station, Berks County",Manager of Software Development – Customer Digital Engineering,Penske Truck Leasing,1971
"Henrietta, Monroe County",IAM Software Engineer II,Rochester Institute of Technology,1972
"De Pere, Brown County",Senior Software Engineer,OPW,1973
"Juno Beach, Palm Beach County",Software Engineer (Frontend),NextEra,1974
"Juno Beach, Palm Beach County",Lead Software Engineer,NextEra,1975
"Houston, Harris County",Software Engineer Coop,Weatherford,1976
"Mooresville, Limestone County","Software Engineer, Senior - GMD Weapons Systems (GWS)",Lockheed Martin,1977
"Redstone Arsenal, Madison County","Software Engineer, Senior - GMD Weapons Systems (GWS)",Lockheed Martin,1978
"Capshaw, Limestone County","Software Engineer, Senior - GMD Weapons Systems (GWS)",Lockheed Martin,1979


In [ ]:
usasalqualdf = df[['Year','Location','Salary Max','Highest Qualification']].copy()
usasalqualdf = usasalqualdf.drop_duplicates(['Year'])
usasalqualdf.rename(columns={'Salary Max': 'SalaryMax','Highest Qualification': 'HighestQualification','Year':'Yearsalqual'}, inplace=True)
usasalqualdf =usasalqualdf.set_index('Yearsalqual')
usasalqualdf=usasalqualdf.dropna()
usasalqualdf.head()

,Location,SalaryMax,HighestQualification
Yearsalqual,,,
1995,"De Pere, Brown County",91064.38,No qualification
1996,"Reading Station, Berks County",73028.40,Level 1-4 certificate
1997,"Reading Station, Berks County",85990.89,Level 5 and 6 diploma
1998,"Keene, Cheshire County",71529.01,Bachelor degree and level 7 qualification
1999,"Bannockburn, Lake County",184085.84,"Post-graduate, honours degree or higher qualif..."


In [ ]:
%%sql sqlite:///./usanzgdp.db3
DROP TABLE IF EXISTS USAQUAL;

Done.


[]

In [ ]:
%%sql sqlite:///./usanzgdp.db3
CREATE TABLE USAQUAL(
    LOCATION VARCHAR(10) NOT NULL,
    SALARYMAX FLOAT NOT NULL,
    HighestQualification VARCHAR(10) NOT NULL,
    Yearsalqual INTEGER NOT NULL,
    PRIMARY KEY (Yearsalqual)
);

CREATE INDEX ix_usaqualtitle_year ON USAQUAL (Yearsalqual);

Done.
Done.


[]

In [ ]:
connection = sqlite3.connect('./usanzgdp.db3')
usasalqualdf.to_sql('USAQUAL', connection, if_exists='append',)
connection.commit()

In [ ]:
connection.close()

In [ ]:
usaqualsqldf = %sql SELECT * FROM USAQUAL
usaqualsqldf

 * sqlite:///./usanzgdp.db3
Done.


LOCATION,SALARYMAX,HighestQualification,Yearsalqual
"De Pere, Brown County",91064.38,No qualification,1995
"Reading Station, Berks County",73028.4,Level 1-4 certificate,1996
"Reading Station, Berks County",85990.89,Level 5 and 6 diploma,1997
"Keene, Cheshire County",71529.01,Bachelor degree and level 7 qualification,1998
"Bannockburn, Lake County",184085.84,"Post-graduate, honours degree or higher qualification",1999
"Reston, Fairfax County",88438.98,No qualification,2000
"Feasterville, Bucks County",108014.34,Level 1-4 certificate,2001
"Bannockburn, Lake County",239122.97,Level 5 and 6 diploma,2002
"De Pere, Brown County",85819.75,Bachelor degree and level 7 qualification,2003
"Harmans, Anne Arundel County",126843.65,"Post-graduate, honours degree or higher qualification",2004


In [ ]:
usafivedf = df[['Year','GOV','TAX','Tax revenue']].copy()
usafivedf = usafivedf.drop_duplicates(['Year'])
usafivedf.rename(columns={'Tax revenue': 'Taxrevenue','Year':'Yearfive'}, inplace=True)
usafivedf =usafivedf.set_index('Yearfive')
usafivedf=usafivedf.dropna()
usafivedf.head()

,GOV,TAX,Taxrevenue
Yearfive,,,
1995,SUPRA,TOTALTAX,Total tax revenue
1996,SUPRA,TOTALTAX,Total tax revenue
1997,SUPRA,TOTALTAX,Total tax revenue
1998,SUPRA,TOTALTAX,Total tax revenue
1999,SUPRA,TOTALTAX,Total tax revenue


In [ ]:
%%sql sqlite:///./usanzgdp.db3
DROP TABLE IF EXISTS USAFIVE;

Done.


[]

In [ ]:
%%sql sqlite:///./usanzgdp.db3
CREATE TABLE USAFIVE(
    GOV VARCHAR(10) NOT NULL,
    TAX VARCHAR(10) NOT NULL,
    TAXREVENUE VARCHAR(10) NOT NULL,
    Yearfive INTEGER NOT NULL,
    PRIMARY KEY (Yearfive)
);

CREATE INDEX ix_usafive_year ON USAFIVE (Yearfive);

Done.
Done.


[]

In [ ]:
connection = sqlite3.connect('./usanzgdp.db3')
usafivedf.to_sql('USAFIVE', connection, if_exists='append',)
connection.commit()

In [ ]:
connection.close()

In [ ]:
usafivesqldf = %sql SELECT * FROM USAFIVE
usafivesqldf

 * sqlite:///./usanzgdp.db3
Done.


GOV,TAX,TAXREVENUE,Yearfive
FED,TOTALTAX,Total tax revenue,1965
FED,TOTALTAX,Total tax revenue,1966
FED,TOTALTAX,Total tax revenue,1967
FED,TOTALTAX,Total tax revenue,1968
FED,TOTALTAX,Total tax revenue,1969
SUPRA,TOTALTAX,Total tax revenue,1970
SUPRA,TOTALTAX,Total tax revenue,1971
SUPRA,TOTALTAX,Total tax revenue,1972
SUPRA,TOTALTAX,Total tax revenue,1973
SUPRA,TOTALTAX,Total tax revenue,1974


In [ ]:
usasixdf = df[['Year','GOV','Highest Qualification','NZD Per Annum']].copy()
usasixdf = usasixdf.drop_duplicates(['Year'])
usasixdf.rename(columns={'Year':'Yearsix','NZD Per Annum': 'NZDPerAnnum','Highest Qualification':'HighestQualification'}, inplace=True)
usasixdf =usasixdf.set_index('Yearsix')
usasixdf=usasixdf.dropna()
usasixdf.head()

,GOV,HighestQualification,NZDPerAnnum
Yearsix,,,
1995,SUPRA,No qualification,62064.0
1996,SUPRA,Level 1-4 certificate,72300.0
1997,SUPRA,Level 5 and 6 diploma,76727.0
1998,SUPRA,Bachelor degree and level 7 qualification,68903.0
1999,SUPRA,"Post-graduate, honours degree or higher qualif...",74809.0


In [ ]:
%%sql sqlite:///./usanzgdp.db3
DROP TABLE IF EXISTS USASIX;

Done.


[]

In [ ]:
%%sql sqlite:///./usanzgdp.db3
CREATE TABLE USASIX(
    HIGHESTQUALIFICATION VARCHAR(10) NOT NULL,
    GOV VARCHAR(10) NOT NULL,
    NZDPERANNUM FLOAT NOT NULL,
    Yearsix INTEGER NOT NULL,
    PRIMARY KEY (Yearsix)
);

CREATE INDEX ix_usasix_year ON USASIX (Yearsix);

Done.
Done.


[]

In [ ]:
connection = sqlite3.connect('./usanzgdp.db3')
usasixdf.to_sql('USASIX', connection, if_exists='append',)
connection.commit()

In [ ]:
connection.close()

In [ ]:
usasixsqldf = %sql SELECT * FROM USASIX
usasixsqldf

 * sqlite:///./usanzgdp.db3
Done.


HIGHESTQUALIFICATION,GOV,NZDPERANNUM,Yearsix
No qualification,SUPRA,62064.0,1995
Level 1-4 certificate,SUPRA,72300.0,1996
Level 5 and 6 diploma,SUPRA,76727.0,1997
Bachelor degree and level 7 qualification,SUPRA,68903.0,1998
"Post-graduate, honours degree or higher qualification",SUPRA,74809.0,1999
No qualification,SUPRA,57188.0,2000
Level 1-4 certificate,SUPRA,61441.0,2001
Level 5 and 6 diploma,SUPRA,66700.0,2002
Bachelor degree and level 7 qualification,SUPRA,65559.0,2003
"Post-graduate, honours degree or higher qualification",SUPRA,66365.0,2004


![image.png](attachment:image.png)

### Selecting Data

Now I'm going to write some SQL statements to select some data.

SQL us fairly simple to understand, but it is a bit picky at times and can create errors, even when you write it correctly. I keep getting incorrect column names for correct column names.

Up first is selecting the table names and putting them into a little list. It's not becuase I've already forgotten the table names. Actually it is, I keep forgetting them. I actually did this first one when I was half way through and thought it was nifty so here you are.

In [ ]:
%%sql

SELECT
    name
FROM
    sqlite_master
WHERE
    type = 'table'
ORDER BY
    name;

 * sqlite:///./usanzgdp.db3
Done.


name
USAFIVE
USAJOBS
USAJOBTITLE
USAQUAL
USASIX
USATAX


### Salary Converted to USD

Below is a select and a join statement, but I've converted the NZD amount to USD. On todays dollar. Would be a lot cooler if I converted it and depricated the dollar amount back to it's value in present day dollars. Maybe next time.

In [ ]:
%%sql
SELECT
    USASIX.NZDPERANNUM,
    USASIX.Yearsix,
    ROUND((USASIX.NZDPERANNUM * 1.62), 1) AS USDnzJOBsalary
FROM
    USASIX
INNER JOIN
    (SELECT
        HighestQualification,
        Location,
        Yearsalqual
    FROM USAQUAL
    WHERE HighestQualification = 'Post-graduate, honours degree or higher qualification') AS Quali
ON USASIX.Yearsix = Quali.Yearsalqual
ORDER BY
    USASIX.Yearsix DESC;

 * sqlite:///./usanzgdp.db3
Done.


NZDPERANNUM,Yearsix,USDnzJOBsalary
59000.0,2014,95580.0
70408.0,2009,114061.0
66365.0,2004,107511.3
74809.0,1999,121190.6


### Now to Euros

This is essentially the same as above, but to Euros.

In [ ]:
%%sql

SELECT
    USASIX.NZDPERANNUM,
    USASIX.Yearsix,
    ROUND((USASIX.NZDPERANNUM * 1.77), 1) AS EURnzJOBsalary
FROM
    USASIX
INNER JOIN
    (SELECT
        HighestQualification,
        Location,
        Yearsalqual
    FROM USAQUAL
    WHERE HighestQualification = 'No qualification') AS Qualino
ON USASIX.Yearsix = Qualino.Yearsalqual
ORDER BY
    USASIX.Yearsix ASC;

 * sqlite:///./usanzgdp.db3
Done.


NZDPERANNUM,Yearsix,EURnzJOBsalary
62064.0,1995,109853.3
57188.0,2000,101222.8
59046.0,2005,104511.4
51300.0,2010,90801.0


### Jobs Paying Top Dollar

Below I'm selecting what is paying lots. Above $120k. Interesting that the job that needs a clearance pays the least. In New Zealand it pays to have a clearance. They are expensive.

In [ ]:
%%sql
SELECT
    JOBTITLE,
    AVG(SalaryMax) AS AvgSalaryMax
FROM
    USAJOBTITLE
INNER JOIN
    USAJOBS
ON
    USAJOBTITLE.LOCATION = USAJOBS.Location AND USAJOBTITLE.Yearjobtitle = USAJOBS.Yearjobs
WHERE
    SalaryMax > 120000
GROUP BY
    JOBTITLE
HAVING
    AVG(SalaryMax) > 140000;

 * sqlite:///./usanzgdp.db3
Done.


JOBTITLE,AvgSalaryMax
Lead Software Engineer - Full Stack,184085.84
Principal Software Engineer,239122.97
Senior Software Engineer - Android Developer,194521.14
Software Engineer Principal - Clearance Required,143159.62


### Selecting Location by Salary

As I want to know where I'm going to get paid the most, I've asked SQL to tell me the average salary of each area, group it by location, ensure the salary is enough to live a good lifestyle, then order it by the average salary that it calculated.

In [ ]:
%%sql
SELECT
    Location,
    AVG(SalaryMax) AS AvgSalaryMax
FROM
    USAJOBS
GROUP BY
    Location
HAVING
    AVG(SalaryMax) > 100000
ORDER BY
    AVG(SalaryMax) ASC;

 * sqlite:///./usanzgdp.db3
Done.


Location,AvgSalaryMax
"Feasterville, Bucks County",104792.456
"New Market, Madison County",109653.7
"Henrietta, Monroe County",110828.35
"Meridianville, Madison County",115651.635
"Capshaw, Limestone County",121152.29000000001
"Priceville, Morgan County",121550.75
"Redstone Arsenal, Madison County",121640.6
"New Hope, Madison County",124066.01
"Gurley, Madison County",124090.11
"Highland Park, Dallas",125283.05


### Seeing Who's Who in Each Particular Zoo

Here I'm selecting some highly qualified job listings by location. I'd also like to know that there's enough people that are qualified in the area. Just to be sure.

In [ ]:
%%sql

SELECT
    LOCATION,
    COUNT(*) AS TotalEmployees,
    SUM(CASE WHEN HighestQualification IN ('Bachelor degree and level 7 qualification', 'Post-graduate, honours degree or higher qualification') THEN 1 ELSE 0 END) AS QualifiedEmployees,
    (SUM(CASE WHEN HighestQualification IN ('Bachelor degree and level 7 qualification', 'Post-graduate, honours degree or higher qualification') THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS PercentageQualified
FROM
    USAQUAL
GROUP BY
    LOCATION
HAVING
    PercentageQualified > 20.0;

 * sqlite:///./usanzgdp.db3
Done.


LOCATION,TotalEmployees,QualifiedEmployees,PercentageQualified
"Bannockburn, Lake County",2,1,50.0
"De Pere, Brown County",2,1,50.0
"Feasterville, Bucks County",4,1,25.0
"Harmans, Anne Arundel County",2,2,100.0
"Highland Park, Dallas",1,1,100.0
"Houston, Harris County",1,1,100.0
"Keene, Cheshire County",1,1,100.0


### Tax Time.

Here I'm asking SQL to calculate tax by the small number in the value column. For this I've multiplied the value column by 10000 and added one, as the rows that were joined are zeros.

I realise this is not actually how much tax each employee is paying, but I could alter this equation in the middle to give me something useful.

In [ ]:
%%sql

SELECT
    LEVELOFGOVERNMENT,
    AVG(VALUE+1 * 10000) AS AvgTaxPerEmployee
FROM
    USATAX
INNER JOIN
    USAFIVE
ON
    USATAX.Yeartax = USAFIVE.Yearfive
GROUP BY
    LEVELOFGOVERNMENT;

 * sqlite:///./usanzgdp.db3
Done.


LEVELOFGOVERNMENT,AvgTaxPerEmployee
Federal or Central government,10000.0
Supranational,10000.367943396228


### Simpler Request

Here is a simpler request from the data.

In [ ]:
%%sql
SELECT * FROM USAJOBS ORDER BY SalaryMax DESC LIMIT 3;

 * sqlite:///./usanzgdp.db3
Done.


Location,SalaryMin,SalaryMax,Yearjobs
"Bannockburn, Lake County",239122.97,239122.97,2002
"Bannockburn, Lake County",194521.14,194521.14,1970
"Bannockburn, Lake County",184085.84,184085.84,1999


### Creating Some Views

In [ ]:
%%sql
CREATE VIEW AllJobsAndTitles AS
SELECT
    USAJOBS.Location,
    USAJOBS.SalaryMin,
    USAJOBS.SalaryMax,
    USAJOBS.Yearjobs,
    USAJOBTITLE.JOBTITLE,
    USAJOBTITLE.COMPANY
FROM
    USAJOBS
INNER JOIN
    USAJOBTITLE
ON
    USAJOBS.Location = USAJOBTITLE.LOCATION AND USAJOBS.Yearjobs = USAJOBTITLE.Yearjobtitle;

 * sqlite:///./usanzgdp.db3
Done.


[]

In [ ]:
%%sql
CREATE VIEW StateGovTaxInfol AS
SELECT
    GOV,
    TAX,
    TAXREVENUE,
    Yearfive
FROM
    USAFIVE
WHERE
    GOV = 'SUPRA';

 * sqlite:///./usanzgdp.db3
Done.


[]

### Testing the views

In [ ]:
%%sql
SELECT * FROM AllJobsAndTitles;

 * sqlite:///./usanzgdp.db3
Done.


Location,SalaryMin,SalaryMax,Yearjobs,JOBTITLE,COMPANY
"Bannockburn, Lake County",194521.14,194521.14,1970,Senior Software Engineer - Android Developer,WALGREENS
"Reading Station, Berks County",104078.76,104078.76,1971,Manager of Software Development – Customer Digital Engineering,Penske Truck Leasing
"Henrietta, Monroe County",110828.35,110828.35,1972,IAM Software Engineer II,Rochester Institute of Technology
"De Pere, Brown County",101236.55,101236.55,1973,Senior Software Engineer,OPW
"Juno Beach, Palm Beach County",92076.77,92076.77,1974,Software Engineer (Frontend),NextEra
"Juno Beach, Palm Beach County",91954.79,91954.79,1975,Lead Software Engineer,NextEra
"Houston, Harris County",63189.18,63189.18,1976,Software Engineer Coop,Weatherford
"Mooresville, Limestone County",130763.43,130763.43,1977,"Software Engineer, Senior - GMD Weapons Systems (GWS)",Lockheed Martin
"Redstone Arsenal, Madison County",121640.6,121640.6,1978,"Software Engineer, Senior - GMD Weapons Systems (GWS)",Lockheed Martin
"Capshaw, Limestone County",121335.3,121335.3,1979,"Software Engineer, Senior - GMD Weapons Systems (GWS)",Lockheed Martin


In [ ]:
%%sql
SELECT * FROM StateGovTaxInfol;

 * sqlite:///./usanzgdp.db3
Done.


GOV,TAX,TAXREVENUE,Yearfive
SUPRA,TOTALTAX,Total tax revenue,1970
SUPRA,TOTALTAX,Total tax revenue,1971
SUPRA,TOTALTAX,Total tax revenue,1972
SUPRA,TOTALTAX,Total tax revenue,1973
SUPRA,TOTALTAX,Total tax revenue,1974
SUPRA,TOTALTAX,Total tax revenue,1975
SUPRA,TOTALTAX,Total tax revenue,1976
SUPRA,TOTALTAX,Total tax revenue,1977
SUPRA,TOTALTAX,Total tax revenue,1978
SUPRA,TOTALTAX,Total tax revenue,1979


### Revised DB Schema

Here is the revised DB Schema after putting some views into it.

I accidentally put a student status view when I was looking into how to create a view from the lecture code.

![image.png](attachment:image.png)

## Conclusion

I think SQL is wonderful. It is simple to use, yet clunky in a way that is pleasant. I would imagine I will use it in the years to come on all sorts of things.

Thanks for the opportunity to present what I've learnt throughout this paper. It has been a fun paper to be a student on.

## References

https://www.youtube.com/watch?v=Hm3JodBR-vs&t=6s

https://www.sqlitetutorial.net/sqlite-python/creating-database/

https://www.geeksforgeeks.org/python-mysql-create-database/

https://www.sqlite.org/lang.html

https://stackoverflow.com/questions/46968294/sqlite-duplicate-column-name

https://stackoverflow.com/questions/11719044/how-to-get-a-float-result-by-dividing-two-integer-values-using-t-sql

https://www.w3schools.com/sql/sql_datatypes.asp

https://www.geeksforgeeks.org/add-column-names-to-dataframe-in-pandas/

https://stackoverflow.com/questions/50885093/how-do-i-remove-rows-with-duplicate-values-of-columns-in-pandas-data-frame

https://www.youtube.com/watch?v=WCSZfmbFJyQ

https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=NZD

https://www.w3schools.com/SQL/sql_view.asp

https://stackoverflow.com/questions/7959963/android-sqlite-cursor-getcolumnindex

https://dbschema.com/

https://dbschema.com/documentation/tutorial.html#layouts

https://www.geeksforgeeks.org/add-column-names-to-dataframe-in-pandas/

https://stackoverflow.com/questions/64491693/keyerror-none-of-are-in-the-columns-when-using-the-python-pandas-set-index

https://stackoverflow.com/questions/6437907/generate-database-schema-using-python